In [166]:
import numpy as np
import pandas as pd
#import matplotlib as plt
import scipy.stats as sts
#import seaborn as sns

In [406]:
# вытаскиваем таблички, которые спарсили ранее

df_spb = pd.read_csv('weather_spb_2017_2023', sep=',')
df_moscow = pd.read_csv('weather_moscow_2017_2023', sep=',')

In [407]:
df_spb.tail()
# поставим колонку с датами в качестве индекса

,Unnamed: 0,Ночная температура,Дневная температура,Влажность,Давление,Направление ветра,Сила ветра,Погодные явления
2120,2023-04-26,10.0,18.0,37.0,752.0,Южный,6.0,малооблачно
2121,2023-04-27,10.0,10.0,69.0,753.0,Восточный,3.0,ясно
2122,2023-04-28,3.0,9.0,81.0,760.0,Западный,7.0,пасмурно
2123,2023-04-29,1.0,7.0,70.0,760.0,Западный,7.0,пасмурно
2124,2023-04-30,1.0,10.0,90.0,757.0,Южный,7.0,пасмурно


In [408]:
df_spb.rename(columns={'Unnamed: 0': 'Дата'}, inplace=True)
df_spb.set_index('Дата', inplace=True)
df_spb.index = pd.to_datetime(df_spb.index)
df_spb.tail()
# починили индекс

,Ночная температура,Дневная температура,Влажность,Давление,Направление ветра,Сила ветра,Погодные явления
Дата,,,,,,,
2023-04-26,10.0,18.0,37.0,752.0,Южный,6.0,малооблачно
2023-04-27,10.0,10.0,69.0,753.0,Восточный,3.0,ясно
2023-04-28,3.0,9.0,81.0,760.0,Западный,7.0,пасмурно
2023-04-29,1.0,7.0,70.0,760.0,Западный,7.0,пасмурно
2023-04-30,1.0,10.0,90.0,757.0,Южный,7.0,пасмурно


In [409]:
# смотрим классы колонок - если в результате преобразований они изменятся, это сигнал, что я поломал данные
# сами колонки самоочевидны и особо комментировать их смысла нет:
# можно обратить внимание на "погодные явления" - это облачность и осадки одновременно, но без количественного показателя
# (осадки в мм на сайте практически везде отсутствуют)
df_spb.dtypes

Ночная температура     float64
Дневная температура    float64
Влажность              float64
Давление               float64
Направление ветра       object
Сила ветра             float64
Погодные явления        object
dtype: object

In [410]:
# смотрим количество пропусков; их не так много
df_spb.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2125 entries, 2017-07-06 to 2023-04-30
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ночная температура   2117 non-null   float64
 1   Дневная температура  2117 non-null   float64
 2   Влажность            2118 non-null   float64
 3   Давление             2118 non-null   float64
 4   Направление ветра    2115 non-null   object 
 5   Сила ветра           2115 non-null   float64
 6   Погодные явления     2118 non-null   object 
dtypes: float64(5), object(2)
memory usage: 132.8+ KB


In [411]:
# посмотрим на значения количественных переменных: если будут аномально большие/маленькие значения - заметим
df_spb.describe()

,Ночная температура,Дневная температура,Влажность,Давление,Сила ветра
count,2117.000000,2117.000000,2118.000000,2118.000000,2115.000000
mean,4.288616,8.212093,74.717658,758.981114,5.284634
std,8.294060,10.112593,17.468281,8.686402,2.130405
min,-22.000000,-20.000000,24.000000,729.000000,1.000000
25%,-1.000000,0.000000,63.000000,753.000000,4.000000
50%,3.000000,7.000000,80.000000,759.000000,5.000000
75%,11.000000,17.000000,87.000000,765.000000,7.000000
max,31.000000,34.000000,100.000000,791.000000,16.000000


In [412]:
# посмотрим на значения переменных object, потому что там могут оказаться кривые данные
# (наличие которых не бросается в глаза в отличие от числовых признаков)

df_spb['Направление ветра'].unique()

array(['Западный', 'Северо-западный', 'Восточный', 'Юго-восточный',
       'Северный', 'Юго-западный', nan, 'Северо-восточный', 'Южный'],
      dtype=object)

In [386]:
df_spb['Погодные явления'].unique()
# здесь есть лишняя абракадабра в виде оставшегося кода элемента, который уберу далее

array(['пасмурно', 'ясно', 'облачно', nan, 'малооблачно', 'дождь',
       'туман', '>\n</img>', 'снег'], dtype=object)

In [420]:
# но на всякий случай проверю, что таких значений немного
df_spb['Погодные явления'].value_counts()
# действительно немного, некриминальная ошибка

Погодные явления
пасмурно       1119
ясно            368
малооблачно     226
снег            160
облачно         131
дождь            93
туман            19
>\n</img>         2
Name: count, dtype: int64

In [400]:
# сейчас будем заполнять пропуски: не средним по всей строке, а средним по месяцу - это будет более точным
# для этого будем итерироваться по месяцам, считать среднее внутри месяца и ставить его в пропуск, если такой есть
# пытался придумать, как это сделать через apply или fillna - не придумал, поэтому придется делать циклом
# в качестве среднего возьмем моду - значения наблюдений слишком разреженные для среднего арифметического
itermonths = pd.date_range(start=df_spb.index[0], end=df_spb.index[-1], freq='M')

In [401]:
for unit in itermonths:
    mask = f'{unit.year}-{unit.month}' # можно изящнее вытащить месяц и год, но мне оказалось быстрее написать это
    piece = df_spb.loc[mask] # месячный срез, по которому итерируемся
    for column in df_spb.columns: # для каждой колонки заполняем пропуски
        df_spb[f'{column}'].fillna(piece[f'{column}'].mode()[0], inplace=True)
        
        # а здесь разбираемся с криво спарсившимся значением
        df_spb['Погодные явления'].replace(
            '>\n</img>', 
            piece['Погодные явления'].mode()[0], 
            inplace=True
        )

In [402]:
df_spb['Направление ветра'].unique()
# пропуски убрали

array(['Западный', 'Северо-западный', 'Восточный', 'Юго-восточный',
       'Северный', 'Юго-западный', 'Северо-восточный', 'Южный'],
      dtype=object)

In [403]:
df_spb['Погодные явления'].unique()
# убрали криво спарсившееся значение

array(['пасмурно', 'ясно', 'облачно', 'малооблачно', 'дождь', 'туман',
       'снег'], dtype=object)

In [404]:
df_spb.info()
# тип данных везде сохранился, пропусков нет 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2125 entries, 2017-07-06 to 2023-04-30
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ночная температура   2125 non-null   float64
 1   Дневная температура  2125 non-null   float64
 2   Влажность            2125 non-null   float64
 3   Давление             2125 non-null   float64
 4   Направление ветра    2125 non-null   object 
 5   Сила ветра           2125 non-null   float64
 6   Погодные явления     2125 non-null   object 
dtypes: float64(5), object(2)
memory usage: 132.8+ KB


In [405]:
# сделаем все то же для Москвы, только в одной ячейке и без комментариев
df_moscow.rename(columns={'Unnamed: 0': 'Дата'}, inplace=True)
df_moscow.set_index('Дата', inplace=True)
df_moscow.index = pd.to_datetime(df_moscow.index)

for unit in itermonths:
    mask = f'{unit.year}-{unit.month}'
    piece = df_moscow.loc[mask]
    for column in df_moscow.columns:
        df_moscow[f'{column}'].fillna(piece[f'{column}'].mode()[0], inplace=True)
        df_moscow['Погодные явления'].replace(
            '>\n</img>', 
            piece['Погодные явления'].mode()[0], 
            inplace=True
        )
        
df_moscow.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2125 entries, 2017-07-06 to 2023-04-30
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ночная температура   2125 non-null   float64
 1   Дневная температура  2125 non-null   float64
 2   Влажность            2125 non-null   float64
 3   Давление             2125 non-null   float64
 4   Направление ветра    2125 non-null   object 
 5   Сила ветра           2125 non-null   float64
 6   Погодные явления     2125 non-null   object 
dtypes: float64(5), object(2)
memory usage: 132.8+ KB


In [421]:
# сохраним полученные таблички и загрузим на гитхаб
df_spb.to_csv('spb_fixed')
df_moscow.to_csv('moscow_fixed')